# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [2]:
!pip install chromadb langchain langchain_community langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.4 MB/s eta 0:00:00

In [3]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [5]:
# If you're using colab, run this
#os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [6]:
llm = OpenAI(temperature=0)

In [25]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "/content/climate_change_speech.txt")

In [31]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="climate_change_speech")

In [32]:
climate_change_speech = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [33]:
from langchain_community.document_loaders import WebBaseLoader

In [34]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [35]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [37]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [38]:
tools = [
    Tool(
        name="Climate Change Speech QA System",
        func=climate_change_speech.run,  # Assuming you have a 'run' method in your system for the new data
        description="Useful for when you need to answer questions about the climate change speech. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="Useful for when you need to answer questions about Ruff (a Python linter). Input should be a fully formed question.",
    ),
]

In [39]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [40]:
# Using the new tool for answering questions related to the climate change speech
agent.invoke(
    "What did the speaker mention about climate change in the speech?"
)




> Entering new AgentExecutor chain...
 I should use the Climate Change Speech QA System to answer this question.
Action: Climate Change Speech QA System
Action Input: "What did the speaker mention about climate change in the speech?"
Observation:  The speaker mentioned that climate change is an existential challenge that threatens our planet, our people, and our way of life. They also mentioned the devastating effects of climate change, the clear link to human activity, and the need for bold, collective action to address it.
Thought: I now know the final answer.
Final Answer: The speaker mentioned that climate change is an existential challenge that threatens our planet, our people, and our way of life. They also mentioned the devastating effects of climate change, the clear link to human activity, and the need for bold, collective action to address it.

> Finished chain.


{'input': 'What did the speaker mention about climate change in the speech?',
 'output': 'The speaker mentioned that climate change is an existential challenge that threatens our planet, our people, and our way of life. They also mentioned the devastating effects of climate change, the clear link to human activity, and the need for bold, collective action to address it.'}

In [41]:
# Using the Ruff QA System tool to answer the question
agent.invoke(
    "Why is Ruff preferred over Flake8 for Python linting?"
)




> Entering new AgentExecutor chain...
 Ruff is a Python linter, so it makes sense to use it for Python linting.
Action: Ruff QA System
Action Input: "Why is Ruff preferred over Flake8 for Python linting?"
Observation:  Ruff is preferred over Flake8 for Python linting because it implements over 800 rules, compared to Flake8's ~409 rules. Additionally, Ruff is capable of automatically fixing its own lint violations and does not require the installation of additional plugins.
Thought: This information is helpful, but I want to know more about Ruff's capabilities.
Action: Ruff QA System
Action Input: "What are some of Ruff's key features?"
Observation:  Some of Ruff's key features include compatibility with Black and Flake8, a comprehensive set of rules for code quality and style, and a powerful formatter that can be used as a drop-in replacement for Black. It also has a user-friendly configuration system and supports editor integration for a seamless coding experience.
Thought: I now ha

{'input': 'Why is Ruff preferred over Flake8 for Python linting?',
 'output': 'Ruff is preferred over Flake8 for Python linting because it has more rules, can automatically fix its own violations, and has a powerful formatter. However, it is only compatible with Python 3 and may have some limitations compared to Flake8.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [49]:
tools = [
    Tool(
        name="Climate Change Speech QA System",
        func=climate_change_speech.run,
        description="Useful for when you need to answer questions about the climate change speech. Input should be a fully formed question, not referencing any obscure pronouns.",
        return_direct=True,  # Directly return the result from the tool
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="Useful for when you need to answer questions about ruff (a Python linter). Input should be a fully formed question.",
        return_direct=True,  # Directly return the result from the tool
    ),
]




In [50]:
# Construct the agent with the updated tool definitions
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)


In [51]:

# Example question regarding the climate change speech, now returning the direct result
result = agent.invoke(
    "What did the president mention about reducing carbon emissions in the climate change speech?"
)






> Entering new AgentExecutor chain...
 The president likely mentioned specific actions or policies related to reducing carbon emissions.
Action: Climate Change Speech QA System
Action Input: "What did the president mention about reducing carbon emissions?"
Observation:  The president mentioned setting new, ambitious targets to cut carbon emissions by 50% by 2030 and achieving net-zero emissions by 2050.


> Finished chain.


In [52]:
# Example query for the Ruff QA System
response_2 = agent.invoke(
    "Why is Ruff preferred over Flake8 for Python linting?"
)



> Entering new AgentExecutor chain...
 Ruff is a newer linter and may have more advanced features.
Action: Ruff QA System
Action Input: "Why is Ruff preferred over Flake8 for Python linting?"
Observation:  Ruff is preferred over Flake8 for Python linting because it implements over 800 rules, compared to Flake8's ~409 rules. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 requires the use of plugins for certain rules. Ruff also supports a wider range of Python versions, including Python 3.13, while Flake8 does not support Python 2. Finally, Ruff's rules are less likely to conflict with each other, making it easier to toggle individual rules on and off.


> Finished chain.


## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [56]:
tools = [
    Tool(
        name="Climate Change Speech QA System",
        func=climate_change_speech.run,
        description="Useful for answering questions related to the recent climate change speech. Input should be a fully formed question, not referencing any obscure pronouns.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="Useful for when you need to answer questions about ruff (a Python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]




In [57]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)



In [58]:
# Now, you can invoke the agent with a new query based on the updated climate change speech.
agent.invoke(
    "What are the key actions proposed in the climate change speech to combat global warming?"
)



> Entering new AgentExecutor chain...
 It would be helpful to use the Climate Change Speech QA System for this question.
Action: Climate Change Speech QA System
Action Input: "What are the key actions proposed in the climate change speech to combat global warming?"
Observation:  The key actions proposed in the climate change speech include reducing emissions, protecting vulnerable communities, and building a sustainable future. It also emphasizes the need for collective action and the role of businesses and communities in finding solutions.


> Finished chain.


{'input': 'What are the key actions proposed in the climate change speech to combat global warming?',
 'output': ' The key actions proposed in the climate change speech include reducing emissions, protecting vulnerable communities, and building a sustainable future. It also emphasizes the need for collective action and the role of businesses and communities in finding solutions.'}